In [ ]:
!pip install sudachipy
!pip install sudachidict_core
!pip install transformers
!pip install tf-keras~=2.16
!pip install japanize-matplotlib

In [ ]:
from sudachipy import tokenizer
from sudachipy import dictionary
from transformers import pipeline


In [ ]:
# Sudachiインスタンス作成
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C  # 詳細モード

In [ ]:
# テキスト
text = "あなたの発言は非常に失礼で、受け入れられないものです。"

In [ ]:
# SudachiPyで形態素解析
tokens = [token.surface() for token in tokenizer_obj.tokenize(text, mode) if token.surface() != ""]
print("形態素解析結果:", tokens)

In [ ]:
# 有害性検出モデル
toxicity_classifier = pipeline("text-classification", model="unitary/toxic-bert")

In [ ]:
# 感情分析モデル Harassment Detector 4で感情と有害性の逆相関をプロットするため、rinnaを加えました。
sentiment_analyzer = pipeline("sentiment-analysis", model="rinna/japanese-roberta-base")

In [ ]:
# スコアリストの準備
toxicity_scores = []
sentiment_scores = []

In [ ]:
# 分割された単語ごとに有害性を検出
# スコアリストの準備
toxicity_scores = []
sentiment_scores = []

# 分割された単語ごとに有害性と感情を検出
for token in tokens:
    # 有害性検出
    toxicity_result = toxicity_classifier(token)
    toxicity_score = toxicity_result[0]['score']
    toxicity_scores.append(toxicity_score)

    # 感情分析
    sentiment_result = sentiment_analyzer(token)
    sentiment_score = sentiment_result[0]['score']
    sentiment_scores.append(sentiment_score)

    print(f"単語: {token}")
    print(f"  有害性スコア: {toxicity_score}")
    print(f"  感情スコア: {sentiment_score}")

In [ ]:
# CSVファイルに書き込み。使いやすいBIにインポートできるようにするため。IBM cognosでの洞察を想定。
import pandas as pd

# DataFrameの作成
results = pd.DataFrame({
    "形態素": tokens,
    "有害性スコア": toxicity_scores,
    "感情スコア": sentiment_scores
})

# DataFrameの表示
print(results)

# CSVファイルとして保存
results.to_csv("text_analysis_results.csv", index=False, encoding='utf-8')
print("結果が 'text_analysis_results.csv' に保存されました。")

In [ ]:
#　X軸: 感情スコア　Y軸: 有害性スコア　各点に形態素をラベルとして表示
# 感情スコアが低い（上位10個）かつ有害性スコアが高いデータを抽出
filtered_data = results.nsmallest(10, '感情スコア').nlargest(10, '有害性スコア')

import matplotlib.pyplot as plt
import matplotlib
import japanize_matplotlib

# プロット
plt.figure(figsize=(12, 8))
plt.scatter(filtered_data['感情スコア'], filtered_data['有害性スコア'], color='red', s=100, alpha=0.7)

# 各点にラベルを表示
for i, row in filtered_data.iterrows():
    plt.text(row['感情スコア'], row['有害性スコア'], row['形態素'], fontsize=9, ha='right')

plt.title('感情スコア vs 有害性スコア (形態素表示)', fontsize=14)
plt.xlabel('感情スコア', fontsize=12)
plt.ylabel('有害性スコア', fontsize=12)
plt.grid(alpha=0.3)
plt.show()

In [ ]:
import seaborn as sns

# 上位スコアに基づくデータ抽出
top_data = filtered_data.sort_values(by='有害性スコア', ascending=False)

# プロット
plt.figure(figsize=(14, 8))
sns.barplot(x='形態素', y='有害性スコア', data=top_data, palette='Reds', label='有害性スコア')
sns.barplot(x='形態素', y='感情スコア', data=top_data, palette='Blues', label='感情スコア')

# グラフの装飾
plt.title('形態素ごとの有害性スコアと感情スコア', fontsize=14)
plt.xlabel('形態素', fontsize=12)
plt.ylabel('スコア', fontsize=12)
plt.xticks(rotation=45)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# バブルサイズを計算
filtered_data['bubble_size'] = filtered_data['感情スコア'] + filtered_data['有害性スコア']

# プロット
plt.figure(figsize=(12, 8))
plt.scatter(
    filtered_data['感情スコア'],
    filtered_data['有害性スコア'],
    s=filtered_data['bubble_size'] * 1000,  # バブルサイズを調整
    alpha=0.5,
    color='purple'
)

# ラベルを追加
for i, row in filtered_data.iterrows():
    plt.text(row['感情スコア'], row['有害性スコア'], row['形態素'], fontsize=9, ha='right')

plt.title('感情スコア vs 有害性スコア (バブルチャート)', fontsize=14)
plt.xlabel('感情スコア', fontsize=12)
plt.ylabel('有害性スコア', fontsize=12)
plt.grid(alpha=0.3)
plt.show()